In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import numpy as np

class Router:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.neighbors = [] 
        self.edges = []  

class Edge:
    def __init__(self, start, end, length):
        self.start = start
        self.end = end
        self.length = length
        self.load = 0  



In [18]:
import pandas as pd
import numpy as np
import random

def generate_sample_data(m, n, num_samples):
    data = {
        'Source': [],
        'Destination': [],
        'Current_Router': [],
        'Action': [],
        'Next_Router': []
    }

    for _ in range(num_samples):
        source = (random.randint(0, m-1), random.randint(0, n-1))
        destination = (random.randint(0, m-1), random.randint(0, n-1))
        current_router = source
        action = random.choice(['down', 'up', 'left', 'right'])

        if action == 'down' and current_router[0] < destination[0]:
            next_router = (current_router[0] + 1, current_router[1])
        elif action == 'up' and current_router[0] > destination[0]:
            next_router = (current_router[0] - 1, current_router[1])
        elif action == 'right' and current_router[1] < destination[1]:
            next_router = (current_router[0], current_router[1] + 1)
        elif action == 'left' and current_router[1] > destination[1]:
            next_router = (current_router[0], current_router[1] - 1)
        else:
            next_router = current_router  

        data['Source'].append(source)
        data['Destination'].append(destination)
        data['Current_Router'].append(current_router)
        data['Action'].append(action)
        data['Next_Router'].append(next_router)

    return pd.DataFrame(data)

def observation(df):
    observations_data = {
        'Source_X': [],
        'Source_Y': [],
        'Destination_X': [],
        'Destination_Y': [],
        'Current_X': [],
        'Current_Y': [],
        'Action':[],
        'Next_X': [],
        'Next_Y': []
    }

    for index, row in df.iterrows():
        source_x, source_y = row['Source']
        dest_x, dest_y = row['Destination']
        current_x, current_y = row['Current_Router']
        action=row['Action']
        next_x, next_y = row['Next_Router'] if row['Action'] == 'send' else (current_x, current_y)

        observations_data['Source_X'].append(source_x)
        observations_data['Source_Y'].append(source_y)
        observations_data['Destination_X'].append(dest_x)
        observations_data['Destination_Y'].append(dest_y)
        observations_data['Current_X'].append(current_x)
        observations_data['Current_Y'].append(current_y)
        observations_data['Action'].append(action)
        observations_data['Next_X'].append(next_x)
        observations_data['Next_Y'].append(next_y)

    return pd.DataFrame(observations_data)


m, n = 5, 5  
num_samples = 20

sample_data = generate_sample_data(m, n, num_samples)

observations_df = observation(sample_data)

print(observations_df)


    Source_X  Source_Y  Destination_X  Destination_Y  Current_X  Current_Y  \
0          3         0              1              4          3          0   
1          1         0              2              0          1          0   
2          0         4              0              0          0          4   
3          3         2              3              2          3          2   
4          2         2              0              2          2          2   
5          2         3              3              3          2          3   
6          4         0              2              3          4          0   
7          3         1              4              1          3          1   
8          1         4              4              0          1          4   
9          3         4              1              3          3          4   
10         3         0              0              4          3          0   
11         0         4              2              1          0 

In [19]:
sample_data.head()

,Source,Destination,Current_Router,Action,Next_Router
0,"(3, 0)","(1, 4)","(3, 0)",up,"(2, 0)"
1,"(1, 0)","(2, 0)","(1, 0)",right,"(1, 0)"
2,"(0, 4)","(0, 0)","(0, 4)",right,"(0, 4)"
3,"(3, 2)","(3, 2)","(3, 2)",left,"(3, 2)"
4,"(2, 2)","(0, 2)","(2, 2)",down,"(2, 2)"
